In [1]:
import numpy as np
import os, sys
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
from collections import Counter
from tqdm.auto import tqdm, trange
from sklearn.preprocessing import MinMaxScaler
import re
import concurrent.futures
from sklearn.metrics import silhouette_score
from scipy.stats import pearsonr
import copy
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, DistributedSampler
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from sdv.single_table import CTGANSynthesizer
from sdv.sampling import Condition
from sdv.evaluation.single_table import get_column_plot

import dask.dataframe as dpd
import dask_geopandas as dgpd
from dask.diagnostics import ProgressBar
from dask.distributed import Client

import warnings
warnings.filterwarnings('ignore')

sys.path.append('../')
from utils.logger import Logger

import gc
gc.collect()

np.random.seed(0)

/home/grads/l/lipai.huang/GAN_simulated_events/se_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = Client(n_workers=100) #192 totally

In [3]:
print(torch.__version__, torch.cuda.is_available())

2.0.1+cu118 True


In [4]:
if torch.cuda.is_available():
    print("CUDA is available. Detected CUDA Devices:")
    for i in range(torch.cuda.device_count()):
        print(f"CUDA Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

CUDA is available. Detected CUDA Devices:
CUDA Device 0: NVIDIA RTX A6000


In [5]:
torch.cuda.set_device(0)

In [6]:
coord_gdf = gpd.read_file('../src/coord/coord_gdf.shp')
coord_gdf = coord_gdf.drop(columns=['cell_rmse1', 'cell_r21', 'cell_rmse2', 'cell_r22', 'depth'])
coord_gdf

,x,y,ter,HUC12,region,channel,geometry
0,2.933766e+06,1.396557e+07,301.388702,Cypress Creek,0,0,"POLYGON ((2934366.000 13964974.635, 2933003.17..."
1,2.934966e+06,1.396557e+07,301.594696,Cypress Creek,0,0,"POLYGON ((2934366.000 13967369.160, 2934380.33..."
2,2.933766e+06,1.396437e+07,294.629181,Cypress Creek,0,0,"POLYGON ((2934366.000 13964974.635, 2934366.00..."
3,2.934966e+06,1.396437e+07,298.529877,Cypress Creek,0,0,"POLYGON ((2935566.000 13963774.635, 2934366.00..."
4,2.936166e+06,1.396437e+07,294.815002,Cypress Creek,0,0,"POLYGON ((2936766.000 13963774.635, 2935566.00..."
...,...,...,...,...,...,...,...
26296,3.039069e+06,1.385008e+07,54.643570,Whiteoak Bayou-Buffalo Bayou,2,1,"POLYGON ((3039427.707 13849492.726, 3038745.86..."
26297,3.039053e+06,1.385088e+07,59.625050,Addicks Reservoir,3,1,"POLYGON ((3039399.212 13851153.541, 3039405.50..."
26298,3.038396e+06,1.385006e+07,60.055576,Whiteoak Bayou-Buffalo Bayou,2,0,"POLYGON ((3038723.769 13850469.724, 3038724.68..."
26299,3.038392e+06,1.385087e+07,59.625050,Addicks Reservoir,3,0,"POLYGON ((3038721.900 13851266.014, 3038723.76..."


In [7]:
coord_union_gdf = gpd.GeoDataFrame(geometry=[coord_gdf.unary_union], crs=coord_gdf.crs)

In [8]:
total_bounds = coord_gdf.total_bounds
total_bounds

array([ 2921166.      , 13769377.      ,  3202957.156923, 13967374.634684])

In [9]:
np.random.seed(3)
sample_event_num = 50
event_indices = np.random.choice(range(593), sample_event_num, replace=False)

In [10]:
event_indices

array([349, 402,  37, 345, 340, 375, 134, 268, 535,  10,  53, 161, 513,
       247, 107, 410, 428, 192, 362, 450, 378,  14, 280,  58, 312, 323,
       237, 438, 467,  48, 288, 328, 130, 137, 131, 239,  45, 194, 592,
       127, 102, 448, 488, 522, 124, 575, 492, 315, 132, 524])

In [37]:
scaler = MinMaxScaler()
xy_scaled = scaler.fit_transform(coord_gdf[['x', 'y']])

def load_and_scale(file_path, scale=False):
    df = pd.read_parquet(file_path)[['x', 'y', 'channel', 'ter', 'cumu_rain', 'peak_int', 'duration', 'depth']]
    if scale:
        df[['x', 'y']] = xy_scaled
    return df
file_paths = [f'../src/tables/data{i}.parquet' for i in event_indices]
selected_events = [load_and_scale(file, scale=True) for file in file_paths]
with ProgressBar():
    result = dpd.concat(selected_events, axis=0)
selected_events_df = result.compute()

In [38]:
selected_events_df = selected_events_df.drop(columns=['channel', 'ter', 'depth'])

In [13]:
selected_events_df['duration'].value_counts()

duration
1     289311
2     210408
3     184107
6     184107
4     105204
7      78903
9      52602
8      52602
11     26301
16     26301
14     26301
5      26301
15     26301
10     26301
Name: count, dtype: int64

In [39]:
selected_events_df['duration'] = selected_events_df['duration'].astype(float)

In [44]:
sample_df = selected_events_df.copy()

In [51]:
def transform_fn_ieq(column_names, data):
    try:
        # Modify peak_int to ensure it's >= cumu_rain / duration
        transformed_data = data.copy()
        transformed_data['ieq'] = data[column_names[0]] / (data[column_names[-1]] + 1e-6)
        transformed_data[column_names[1]] = transformed_data[[column_names[1], 'ieq']].max(axis=1)
        data[column_names[1]] = transformed_data[column_names[1]]
        return data
    except Exception as e:
        print(f"Error in transform_fn_ieq: {e}")
        raise

In [52]:
transform_fn_ieq(['cumu_rain', 'peak_int', 'duration'], sample_df)

,x,y,cumu_rain,peak_int,duration
0,0.042735,1.000000,0.956980,0.845267,3.0
1,0.047009,1.000000,0.978379,0.869415,3.0
2,0.042735,0.993865,1.022239,0.908600,3.0
3,0.047009,0.993865,1.063435,0.956885,3.0
4,0.051282,0.993865,1.104630,1.005170,3.0
...,...,...,...,...,...
26296,0.417746,0.409514,11.554805,8.105234,6.0
26297,0.417691,0.413603,11.665516,8.170113,6.0
26298,0.415349,0.409447,11.400763,8.008066,6.0
26299,0.415335,0.413595,11.522994,8.081010,6.0


# cCTGAN events augmentation

In [40]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(selected_events_df)

In [41]:
metadata

{
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1",
    "columns": {
        "x": {
            "sdtype": "numerical"
        },
        "y": {
            "sdtype": "numerical"
        },
        "cumu_rain": {
            "sdtype": "numerical"
        },
        "peak_int": {
            "sdtype": "numerical"
        },
        "duration": {
            "sdtype": "numerical"
        }
    }
}

In [42]:
lr_sets = [
    # G, D
    [1e-5, 1e-5],
    [2e-5, 1e-5],
    [1e-4, 1e-4],
    [2e-4, 1e-4],
    [1e-3, 1e-3],
    [2e-3, 1e-3]
]

In [43]:
x_bounds_constraints = {
    'constraint_class': 'ScalarRange',
    'constraint_parameters': {
        'column_name': 'x',
        'low_value': 0.0,
        'high_value': 1.0,
        'strict_boundaries': False
    }
}

y_bounds_constraints = {
    'constraint_class': 'ScalarRange',
    'constraint_parameters': {
        'column_name': 'y',
        'low_value': 0.0,
        'high_value': 1.0,
        'strict_boundaries': False
    }
}

peak_int_constraints = {
    'constraint_class': 'PeakIntConstraintClass',
    'constraint_parameters': {
        'column_names': ['cumu_rain', 'peak_int', 'duration']
    }
}

positive_constraints = {
    'constraint_class': 'Positive',
    'constraint_parameters': {
        'column_name': 'peak_int',
        'strict_boundaries': False
    }
}

inequalty_constraints = {
    'constraint_class': 'Inequality',
    'constraint_parameters': {
        'low_column_name': 'peak_int',
        'high_column_name': 'cumu_rain'
    }
}

In [19]:
def filter_rows_by_condition(df):
    return df[(df['peak_int'] >= df['cumu_rain'] / df['duration']) & (df['cumu_rain'] >= df['peak_int'])]

In [75]:
ctgan_logger = Logger('../logs/cCTGAN_model.log')

In [76]:
for lr_id, lrs in tqdm(enumerate(lr_sets), total=len(lr_sets)):
    for epoch in range(50, 301, 50):
        try:
            ctgan_logger.log_info(f'[{lr_id+1}/{len(lr_sets)}] [Epoch: {epoch}]: {lrs[0]}_{lrs[1]}_{epoch+1}')
            ctgan_synthesizer = CTGANSynthesizer(metadata, epochs=epoch, 
                                                cuda=True, verbose=True, enforce_rounding=False, 
                                                batch_size=500, generator_lr=lrs[0], discriminator_lr=lrs[1])
            ctgan_synthesizer.load_custom_constraint_classes(
                filepath = '../models/cCTGAN.py',
                class_names = ['PeakIntConstraintClass']
            )
            ctgan_synthesizer.add_constraints(
                constraints = 
                [peak_int_constraints]
                + [x_bounds_constraints, y_bounds_constraints, inequalty_constraints, positive_constraints]
                
            )
            # train ctgan
            ctgan_logger.log_info('Start training...')
            ctgan_synthesizer.fit(selected_events_df)
            ctgan_synthesizer.save(f'../checkpoints/cCTGAN/{lrs[0]}_{lrs[1]}_{epoch+1}.pkl')
            ctgan_logger.log_info('Generating events...')
            filtered_df_path = f'../outputs/augmented_data/{lrs[0]}_{lrs[1]}_{epoch}.parquet'
            ctgan_synthetic_df = ctgan_synthesizer.sample(num_rows=5000000, batch_size=500)
            ctgan_synthetic_df[['x', 'y']] = scaler.inverse_transform(ctgan_synthetic_df[['x', 'y']])
            ctgan_synthetic_gdf = gpd.GeoDataFrame(
                ctgan_synthetic_df, 
                geometry=gpd.points_from_xy(ctgan_synthetic_df.x, ctgan_synthetic_df.y),
                crs=coord_gdf.crs
            )
            ctgan_logger.log_info('Filtering...')
            ctgan_synthetic_inbound_df = ctgan_synthetic_gdf.sjoin(coord_union_gdf, predicate='within').drop(columns=['geometry', 'index_right'])
            ctgan_synthetic_inbound_df = filter_rows_by_condition(ctgan_synthetic_inbound_df).reset_index(drop=True)
            ctgan_synthetic_inbound_df.to_parquet(filtered_df_path)
        except Exception as e:
            ctgan_logger.log_error(e)

  0%|          | 0/6 [00:00<?, ?it/s]

Index(['x', 'y', 'cumu_rain', 'peak_int', 'duration'], dtype='object')
Index(['x', 'y', 'cumu_rain', 'peak_int', 'duration'], dtype='object')
Index(['x', 'y', 'cumu_rain', 'peak_int', 'duration'], dtype='object')


  0%|          | 0/6 [00:43<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
ctgan_synthesizer.get_constraints()